In [29]:
!pip install datacontract-cli
!pip install pyspark
!uv tool install --python python3.11 --upgrade 'datacontract-cli[all]'

Resolved 172 packages in 2m 42s                                      
⠙ Preparing packages... (0/172)                                                 
⠙ Preparing packages... (0/172)------------     0 B/452.71 KiB          
⠙ Preparing packages... (0/172)------------     0 B/452.71 KiB          
py4j                 ------------------------------     0 B/198.25 KiB
⠙ Preparing packages... (0/172)------------     0 B/452.71 KiB          
py4j                 ------------------------------     0 B/198.25 KiB
⠙ Preparing packages... (0/172)------------     0 B/452.71 KiB          
py4j                 ------------------------------     0 B/198.25 KiB
⠙ Preparing packages... (0/172)------------     0 B/452.71 KiB          
oauthlib             ------------------------------     0 B/156.31 KiB
py4j                 ------------------------------     0 B/198.25 KiB
⠙ Preparing packages... (0/172)------------     0 B/452.71 KiB          
oauthlib             ------------------------------     

In [32]:
!pip uninstall datacontract -y

Found existing installation: datacontract 0.1.0
Uninstalling datacontract-0.1.0:
  Successfully uninstalled datacontract-0.1.0


In [15]:
import os

In [16]:
def get_absolute_path(*relative_parts):

    if 'dbutils' in globals():
        # dbutils is available to the entry point when run in Databricks
        base_dir = os.path.dirname(dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()) # type: ignore
        path = os.path.normpath(os.path.join(base_dir, *relative_parts))
        return path if path.startswith("/Workspace") else "/Workspace" + path
    else:
        return os.path.join(*relative_parts)

In [33]:
from datacontract.data_contract import DataContract
import inspect
import yaml
   
data_contract_file = os.path.abspath(
    os.path.join(os.getcwd(), "..", "datacontract.yaml")
)
with open(data_contract_file, "r") as f:
    contract_yaml = yaml.safe_load(f)

data_contract = DataContract(
    schema=contract_yaml["models"],
    name=contract_yaml["info"]["title"],
    version=contract_yaml["info"]["version"],
)

sql_ddl = data_contract.export(export_format="sql")
sql_ddl

ModuleNotFoundError: No module named 'datacontract.data_contract'; 'datacontract' is not a package

In [24]:
import subprocess

subprocess.run(
    ["datacontract", "export", "../datacontract.yaml", "--format", "sql"],
    check=True,
)

-- Data Contract: urn:datacontract:s231871:berlin-crimes
-- SQL Dialect: databricks
CREATE OR REPLACE TABLE acme.berlin-crimes.articles (
  Year BIGINT not null COMMENT "The year from which the data was collected",
  District STRING not null COMMENT "The district from which the data was collected",
  Code BIGINT not null COMMENT "The District Code from which the data was collected so it is a more fine grained representation for the District",
  Location STRING not null COMMENT "The location from which the data was collected",
  Robbery BIGINT not null COMMENT "The number of robberies in the District",
  Street_robbery BIGINT not null COMMENT "The number of street robberies in the District",
  Injury BIGINT not null COMMENT "The number of injuries in the District"
) COMMENT "One record per Code in district with";


CompletedProcess(args=['datacontract', 'export', '../datacontract.yaml', '--format', 'sql'], returncode=0)

In [25]:
from helpers.spark import get_spark

spark = get_spark()
spark.sql(sql_ddl)

NameError: name 'sql_ddl' is not defined